In [1]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
import pandas as pd
import numpy as np
import utils.public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import (
    pack_padded_sequence,
    pack_sequence,
    unpack_sequence,
    unpad_sequence,
)
import torch
from tqdm.notebook import trange, tqdm
import torch.nn as nn
import torch.optim as optim
import wandb
import utils.torch_classes
from utils.model_saver import model_saver_wandb as model_saver
import utils.training_testing
from itertools import combinations
import gc
from sklearn.decomposition import PCA
import sys
import lightgbm as lgb
import time
from sklearn.preprocessing import OneHotEncoder

In [2]:
%env "WANDB_NOTEBOOK_NAME" os.path.basename(__file__)

env: "WANDB_NOTEBOOK_NAME"=os.path.basename(__file__)


In [3]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [4]:
torch.__version__

'2.1.1'

In [5]:
if torch.cuda.is_available():
    device = torch.device(
        "cuda:0"
    )  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [6]:
train = pd.read_csv("data/train.csv")
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [7]:
import importlib

In [8]:
lgbm_columns = [
    "stock_id",
    "seconds_in_bucket",
    "imbalance_size",
    "imbalance_buy_sell_flag",
    "reference_price",
    "matched_size",
    "far_price",
    "near_price",
    "bid_price",
    "bid_size",
    "ask_price",
    "ask_size",
    "wap",
    "overall_medvol",
    "first5min_medvol",
    "last5min_medvol",
    "bid_plus_ask_sizes",
    "imbalance_ratio",
    "imb_s1",
    "imb_s2",
    "ask_x_size",
    "bid_x_size",
    "ask_minus_bid",
    "bid_price_over_ask_price",
    "reference_price_minus_far_price",
    "reference_price_times_far_price",
    "reference_price_times_near_price",
    "reference_price_minus_ask_price",
    "reference_price_times_ask_price",
    "reference_price_ask_price_imb",
    "reference_price_minus_bid_price",
    "reference_price_times_bid_price",
    "reference_price_bid_price_imb",
    "reference_price_minus_wap",
    "reference_price_times_wap",
    "reference_price_wap_imb",
    "far_price_minus_near_price",
    "far_price_times_near_price",
    "far_price_minus_ask_price",
    "far_price_times_ask_price",
    "far_price_minus_bid_price",
    "far_price_times_bid_price",
    "far_price_times_wap",
    "far_price_wap_imb",
    "near_price_minus_ask_price",
    "near_price_times_ask_price",
    "near_price_ask_price_imb",
    "near_price_minus_bid_price",
    "near_price_times_bid_price",
    "near_price_bid_price_imb",
    "near_price_minus_wap",
    "near_price_wap_imb",
    "ask_price_minus_bid_price",
    "ask_price_times_bid_price",
    "ask_price_minus_wap",
    "ask_price_times_wap",
    "ask_price_wap_imb",
    "bid_price_minus_wap",
    "bid_price_times_wap",
    "bid_price_wap_imb",
    "reference_price_far_price_near_price_imb2",
    "reference_price_far_price_ask_price_imb2",
    "reference_price_far_price_bid_price_imb2",
    "reference_price_far_price_wap_imb2",
    "reference_price_near_price_ask_price_imb2",
    "reference_price_near_price_bid_price_imb2",
    "reference_price_near_price_wap_imb2",
    "reference_price_ask_price_bid_price_imb2",
    "reference_price_ask_price_wap_imb2",
    "reference_price_bid_price_wap_imb2",
    "far_price_near_price_ask_price_imb2",
    "far_price_near_price_bid_price_imb2",
    "far_price_near_price_wap_imb2",
    "far_price_ask_price_bid_price_imb2",
    "far_price_ask_price_wap_imb2",
    "far_price_bid_price_wap_imb2",
    "near_price_ask_price_bid_price_imb2",
    "near_price_ask_price_wap_imb2",
    "near_price_bid_price_wap_imb2",
    "ask_price_bid_price_wap_imb2",
    "pca_prices",
]

weights = [
    0.004,
    0.001,
    0.002,
    0.006,
    0.004,
    0.004,
    0.002,
    0.006,
    0.006,
    0.002,
    0.002,
    0.008,
    0.006,
    0.002,
    0.008,
    0.006,
    0.002,
    0.006,
    0.004,
    0.002,
    0.004,
    0.001,
    0.006,
    0.004,
    0.002,
    0.002,
    0.004,
    0.002,
    0.004,
    0.004,
    0.001,
    0.001,
    0.002,
    0.002,
    0.006,
    0.004,
    0.004,
    0.004,
    0.006,
    0.002,
    0.002,
    0.04,
    0.002,
    0.002,
    0.004,
    0.04,
    0.002,
    0.001,
    0.006,
    0.004,
    0.004,
    0.006,
    0.001,
    0.004,
    0.004,
    0.002,
    0.006,
    0.004,
    0.006,
    0.004,
    0.006,
    0.004,
    0.002,
    0.001,
    0.002,
    0.004,
    0.002,
    0.008,
    0.004,
    0.004,
    0.002,
    0.004,
    0.006,
    0.002,
    0.004,
    0.004,
    0.002,
    0.004,
    0.004,
    0.004,
    0.001,
    0.002,
    0.002,
    0.008,
    0.02,
    0.004,
    0.006,
    0.002,
    0.02,
    0.002,
    0.002,
    0.006,
    0.004,
    0.002,
    0.001,
    0.02,
    0.006,
    0.001,
    0.002,
    0.004,
    0.001,
    0.002,
    0.006,
    0.006,
    0.004,
    0.006,
    0.001,
    0.002,
    0.004,
    0.006,
    0.006,
    0.001,
    0.04,
    0.006,
    0.002,
    0.004,
    0.002,
    0.002,
    0.006,
    0.002,
    0.002,
    0.004,
    0.006,
    0.006,
    0.002,
    0.002,
    0.008,
    0.006,
    0.004,
    0.002,
    0.006,
    0.002,
    0.004,
    0.006,
    0.002,
    0.004,
    0.001,
    0.004,
    0.002,
    0.004,
    0.008,
    0.006,
    0.008,
    0.002,
    0.004,
    0.002,
    0.001,
    0.004,
    0.004,
    0.004,
    0.006,
    0.008,
    0.004,
    0.001,
    0.001,
    0.002,
    0.006,
    0.004,
    0.001,
    0.002,
    0.006,
    0.004,
    0.006,
    0.008,
    0.002,
    0.002,
    0.004,
    0.002,
    0.04,
    0.002,
    0.002,
    0.004,
    0.002,
    0.002,
    0.006,
    0.02,
    0.004,
    0.002,
    0.006,
    0.02,
    0.001,
    0.002,
    0.006,
    0.004,
    0.006,
    0.004,
    0.004,
    0.004,
    0.004,
    0.002,
    0.004,
    0.04,
    0.002,
    0.008,
    0.002,
    0.004,
    0.001,
    0.004,
    0.006,
    0.004,
]

In [9]:
weights_df = pd.DataFrame(
    data=list(zip(range(0, 201), weights)), columns=["stock_id", "index_weight"]
)

In [10]:
train = train.merge(weights_df, on="stock_id")

In [11]:
train["wap_calc"] = (
    train["bid_price"] * train["ask_size"] + train["ask_price"] * train["bid_size"]
) / (train["ask_size"] + train["bid_size"])

In [12]:
train.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id', 'index_weight',
       'wap_calc'],
      dtype='object')

In [13]:
def generate_prev_race(df_in, df_g, rolling_window=10, factor=""):
    df = df_in.copy()
    original_cols = df_in.columns
    df[f"wap_t-60"] = df_g["wap"].shift(6)
    df[f"target_t-60"] = df_g["target"].shift(6)
    df[f"initial_wap"] = df_g["wap_calc"].transform("first")
    df[f"initial_bid_size"] = df_g["bid_size"].transform("first")
    df[f"initial_ask_size"] = df_g["ask_size"].transform("first")
    cols = ["bid_price", "ask_price", "bid_size", "ask_size", "wap"]
    for i in cols:
        df[f"{i}_t-60"] = df_g[i].shift(-6)
    for i in cols:
        df[f"{i}_t10"] = df_g[i].shift(1)

    return df

In [14]:
def generate_index(df_in, df_g, rolling_window=10, factor=""):
    df = df_in.copy()
    df[f"index_wap"] = df_g["wap_weighted"].transform("mean")
    return df


def generate_index_2(df_in, df_g, rolling_window=10, factor=""):
    df = df_in.copy()
    df[f"index_wap_t-60"] = df_g["index_wap"].shift(6)
    df[f"index_wap_init"] = df_g["index_wap"].transform("first")
    return df


def generate_index_3(df_in, df_g, rolling_window=10, factor=""):
    df = df_in.copy()
    df[f"index_wap_t-60"] = df_g["index_wap_move_to_init"].shift(6)
    return df

In [15]:
train["wap_weighted"] = train["wap"] * train["index_weight"]
train_g = train.groupby(["stock_id", "date_id"])
train = generate_prev_race(train, train_g)
train["delta_wap"] = train["wap"] / train["wap_t-60"]

train_g = train.groupby(["seconds_in_bucket", "date_id"])
train = generate_index(train, train_g)


train["wap_move_to_init"] = train["wap_calc"] / train["initial_wap"]
train_g = train.groupby(["date_id"])
train = generate_index_2(train, train_g)

train["index_wap_move_to_init"] = train["index_wap"] / train["index_wap_init"]
train_g = train.groupby(["date_id"])
train = generate_index_3(train, train_g)

In [16]:
train.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id', 'index_weight',
       'wap_calc', 'wap_weighted', 'wap_t-60', 'target_t-60', 'initial_wap',
       'initial_bid_size', 'initial_ask_size', 'bid_price_t-60',
       'ask_price_t-60', 'bid_size_t-60', 'ask_size_t-60', 'bid_price_t10',
       'ask_price_t10', 'bid_size_t10', 'ask_size_t10', 'wap_t10', 'delta_wap',
       'index_wap', 'wap_move_to_init', 'index_wap_t-60', 'index_wap_init',
       'index_wap_move_to_init'],
      dtype='object')

In [17]:
train["target_calc"] = (
    -(
        (train["wap_t-60"] / train["wap"])
        - (train["index_wap_t-60"] / train["index_wap_move_to_init"])
    )
    * 10000
)
train["target_delta"] = train["target_t-60"] - train["target_calc"]

In [18]:
train_stock_0 = train[train["stock_id"] == 0].dropna(subset="bid_size_t-60").copy()
train_stock_0.head(20)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,ask_size_t10,wap_t10,delta_wap,index_wap,wap_move_to_init,index_wap_t-60,index_wap_init,index_wap_move_to_init,target_calc,target_delta
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,...,NaN,NaN,0.999483,0.005031,1.000000,NaN,0.005031,1.000000,NaN,NaN
1,0,0,10,1299772.70,1,1.000026,15261106.63,NaN,NaN,0.999812,...,8493.03,1.000000,0.999471,0.005033,0.999892,NaN,0.005031,1.000356,NaN,NaN
2,0,0,20,1299772.70,1,0.999919,15261106.63,NaN,NaN,0.999812,...,23519.16,0.999892,0.999694,0.005034,0.999842,NaN,0.005031,1.000525,NaN,NaN
3,0,0,30,1299772.70,1,1.000133,15261106.63,NaN,NaN,1.000026,...,12131.60,0.999842,0.999659,0.005034,1.000085,NaN,0.005031,1.000547,NaN,NaN
4,0,0,40,1218204.43,1,1.000455,15342674.90,NaN,NaN,1.000241,...,46203.30,1.000085,1.000056,0.005035,1.000317,NaN,0.005031,1.000635,NaN,NaN
5,0,0,50,1218204.43,1,1.000455,15342674.90,NaN,NaN,1.000348,...,26610.45,1.000317,1.000392,0.005035,1.000435,NaN,0.005031,1.000668,NaN,NaN
6,0,0,60,1218204.43,1,1.000562,15342674.90,NaN,NaN,1.000455,...,9897.22,1.000434,1.000622,0.005036,1.000517,1.000000,0.005031,1.000815,-1.930193,-1.099511
7,0,0,70,1264494.89,1,1.000455,15352380.96,NaN,NaN,1.000348,...,10085.04,1.000517,1.000459,0.005036,1.000422,1.000356,0.005031,1.000843,-0.279044,0.668858
8,0,0,80,1189832.86,1,1.000241,15427043.00,NaN,NaN,1.000133,...,17366.82,1.000421,1.000322,0.005033,1.000148,1.000525,0.005031,1.000409,4.376209,-0.156200
9,0,0,90,1189272.89,1,1.000562,15427602.97,NaN,NaN,1.000348,...,61984.40,1.000148,1.000639,0.005033,1.000427,1.000547,0.005031,1.000345,8.409505,-2.959257


In [19]:
train[
    [
        "seconds_in_bucket",
        "imbalance_size",
        "imbalance_buy_sell_flag",
        "reference_price",
        "matched_size",
        "far_price",
        "near_price",
        "bid_price",
        "bid_size",
        "ask_price",
        "ask_size",
        "wap",
        "index_weight",
        "wap_calc",
        "initial_wap",
        "wap_weighted",
        "index_wap",
        "index_wap_init",
        "index_wap_move_to_init",
    ]
].isna().sum()

seconds_in_bucket                0
imbalance_size                 220
imbalance_buy_sell_flag          0
reference_price                220
matched_size                   220
far_price                  2894342
near_price                 2857180
bid_price                      220
bid_size                         0
ask_price                      220
ask_size                         0
wap                            220
index_weight                     0
wap_calc                       220
initial_wap                    220
wap_weighted                   220
index_wap                        0
index_wap_init                   0
index_wap_move_to_init           0
dtype: int64

In [20]:
# train_stock_0.to_csv('train_with_new_vars_0stock.csv')

In [21]:
median_vol = pd.read_csv("archive/MedianVolV2.csv")
median_vol.index.name = "stock_id"
median_vol = median_vol[["overall_medvol", "first5min_medvol", "last5min_medvol"]]
median_sizes = (
    train.groupby("stock_id")["bid_size"].median()
    + train.groupby("stock_id")["ask_size"].median()
)
std_sizes = (
    train.groupby("stock_id")["bid_size"].median()
    + train.groupby("stock_id")["ask_size"].median()
)

In [22]:
train["bid_price_target"] = train["bid_price"] - train["bid_price_t-60"]
train["bid_price_t-60"] = train["bid_price_target"] * 10_000

In [23]:
train["wap_target"] = train["wap"] - train["wap_t-60"]
train["wap_price_t-60"] = train["wap_target"] * 10_000

In [24]:
targets = ["wap", "bid_price", "ask_price"]
for i in targets:
    train[f"{i}_prev_move"] = (train[f"{i}"] - train[f"{i}_t10"]).fillna(0) * 10000

In [25]:
train["ask_price_target"] = train["ask_price"] - train["ask_price_t-60"]
train["ask_price_t-60"] = train["ask_price_target"] * 10_000

In [26]:
train[["bid_price_t-60", "bid_price"]].head(10)

,bid_price_t-60,bid_price
0,-6.43,0.999812
1,-5.36,0.999812
2,-3.21,0.999812
3,-3.22,1.000026
4,0.00,1.000241
5,3.22,1.000348
6,6.43,1.000455
7,5.36,1.000348
8,4.28,1.000133
9,6.43,1.000348


In [27]:
def feat_eng(df):
    cols = [c for c in df.columns if c not in ["row_id", "time_id"]]
    df = df[cols]
    df = df.merge(median_vol, how="left", left_on="stock_id", right_index=True)

    df["bid_plus_ask_sizes"] = df["bid_size"] + train["ask_size"]
    #     df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df["std_size"] = df["stock_id"].map(std_sizes.to_dict())
    #     df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0)
    df["imbalance_ratio"] = df["imbalance_size"] / df["matched_size"]

    df["imb_s1"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["imb_s2"] = df.eval(
        "(imbalance_size-matched_size)/(matched_size+imbalance_size)"
    )

    df["ask_x_size"] = df.eval("ask_size*ask_price")
    df["bid_x_size"] = df.eval("bid_size*bid_price")

    df["ask_minus_bid"] = df["ask_x_size"] - df["bid_x_size"]

    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])

    prices = [
        "reference_price",
        "far_price",
        "near_price",
        "ask_price",
        "bid_price",
        "wap",
    ]

    for c in combinations(prices, 2):
        df[f"{c[0]}_minus_{c[1]}"] = (df[f"{c[0]}"] - df[f"{c[1]}"]).astype(np.float32)
        df[f"{c[0]}_times_{c[1]}"] = (df[f"{c[0]}"] * df[f"{c[1]}"]).astype(np.float32)
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]}-{c[1]})/({c[0]}+{c[1]})")

    for c in combinations(prices, 3):
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1) - min_ - max_

        df[f"{c[0]}_{c[1]}_{c[2]}_imb2"] = (max_ - mid_) / (mid_ - min_)

    df.drop(
        columns=[
            # 'date_id',
            "reference_price_far_price_imb",
            "reference_price_minus_near_price",
            "reference_price_near_price_imb",
            "far_price_near_price_imb",
            "far_price_ask_price_imb",
            "far_price_bid_price_imb",
            "far_price_minus_wap",
            "std_size",
            "bid_size_over_ask_size",
            "ask_price_bid_price_imb",
            "near_price_times_wap",
        ],
        inplace=True,
    )

    # gc.collect()

    df.replace([np.inf, -np.inf], 0, inplace=True)

    return df

In [28]:
y = train["target"].values
X = feat_eng(train)
prices = [
    c for c in X.columns if ("price" in c) and ("target" not in c) and ("60" not in c)
]
print(prices)

['reference_price', 'far_price', 'near_price', 'bid_price', 'ask_price', 'bid_price_t10', 'ask_price_t10', 'bid_price_prev_move', 'ask_price_prev_move', 'bid_price_over_ask_price', 'reference_price_minus_far_price', 'reference_price_times_far_price', 'reference_price_times_near_price', 'reference_price_minus_ask_price', 'reference_price_times_ask_price', 'reference_price_ask_price_imb', 'reference_price_minus_bid_price', 'reference_price_times_bid_price', 'reference_price_bid_price_imb', 'reference_price_minus_wap', 'reference_price_times_wap', 'reference_price_wap_imb', 'far_price_minus_near_price', 'far_price_times_near_price', 'far_price_minus_ask_price', 'far_price_times_ask_price', 'far_price_minus_bid_price', 'far_price_times_bid_price', 'far_price_times_wap', 'far_price_wap_imb', 'near_price_minus_ask_price', 'near_price_times_ask_price', 'near_price_ask_price_imb', 'near_price_minus_bid_price', 'near_price_times_bid_price', 'near_price_bid_price_imb', 'near_price_minus_wap', 'n

In [29]:
prices = [
    c for c in X.columns if ("price" in c) and ("target" not in c) and ("60" not in c)
]
# prices = [c for c in train.columns if 'price' in c]
pca_prices = PCA(n_components=1)
X["pca_prices"] = pca_prices.fit_transform(X[prices].fillna(1))

In [30]:
X.dtypes

stock_id                                 int64
date_id                                  int64
seconds_in_bucket                        int64
imbalance_size                         float64
imbalance_buy_sell_flag                  int64
                                        ...   
near_price_ask_price_bid_price_imb2    float64
near_price_ask_price_wap_imb2          float64
near_price_bid_price_wap_imb2          float64
ask_price_bid_price_wap_imb2           float64
pca_prices                             float64
Length: 115, dtype: object

In [31]:
lgbm = lgb.Booster(model_file="data/lgbm_model_new_t60.lgb")
X_train = X[[c for c in X.columns if ("target" not in c) and ("60" not in c)]].drop(
    columns=["delta_wap", "date_id"]
)
lgbm_preds = lgbm.predict(X_train)
X["lgbm_preds"] = lgbm_preds

del pca_prices

In [32]:
# X = X.join(pca)
X = X.dropna(subset="wap_t-60").reset_index()

In [33]:
del train
# stop

In [34]:
stat_cols = [
    "seconds_in_bucket",
    "imbalance_size",
    "imbalance_buy_sell_flag",
    "reference_price",
    "matched_size",
    "bid_price",
    "bid_size",
    "ask_price",
    "ask_size",
    "wap",
    "index_weight",
    "wap_calc",
    "initial_wap",
    "wap_weighted",
    "index_wap",
    "index_wap_init",
    "index_wap_move_to_init",
    "wap_prev_move",
    "bid_price_prev_move",
    "ask_price_prev_move",
    "pca_prices",
    "lgbm_preds",
]

# stat_cols = ['seconds_in_bucket', 'imbalance_size',
#        'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
#     #    'far_price', 'near_price',


#        'bid_price', 'bid_size', 'ask_price', 'ask_size',
#         'wap', 'index_weight','wap_calc','initial_wap','wap_weighted', 'index_wap', 'index_wap_init', 'index_wap_move_to_init',

#         'wap_prev_move','bid_price_prev_move','ask_price_prev_move',

#                  'overall_medvol', 'first5min_medvol',
#        'last5min_medvol', 'bid_plus_ask_sizes', 'imbalance_ratio', 'imb_s1',
#        'imb_s2', 'ask_x_size', 'bid_x_size', 'ask_minus_bid',
#        'bid_price_over_ask_price', 'reference_price_minus_far_price',
#        'reference_price_times_far_price', 'reference_price_times_near_price',
#        'reference_price_minus_ask_price', 'reference_price_times_ask_price',
#        'reference_price_ask_price_imb', 'reference_price_minus_bid_price',
#        'reference_price_times_bid_price', 'reference_price_bid_price_imb',
#        'reference_price_minus_wap', 'reference_price_times_wap',
#        'reference_price_wap_imb', 'far_price_minus_near_price',
#        'far_price_times_near_price', 'far_price_minus_ask_price',
#        'far_price_times_ask_price', 'far_price_minus_bid_price',
#        'far_price_times_bid_price', 'far_price_times_wap', 'far_price_wap_imb',
#        'near_price_minus_ask_price', 'near_price_times_ask_price',
#        'near_price_ask_price_imb', 'near_price_minus_bid_price',
#        'near_price_times_bid_price', 'near_price_bid_price_imb',
#        'near_price_minus_wap', 'near_price_wap_imb',
#        'ask_price_minus_bid_price', 'ask_price_times_bid_price',
#        'ask_price_minus_wap', 'ask_price_times_wap', 'ask_price_wap_imb',
#        'bid_price_minus_wap', 'bid_price_times_wap', 'bid_price_wap_imb',
#        'reference_price_far_price_near_price_imb2',
#        'reference_price_far_price_ask_price_imb2',
#        'reference_price_far_price_bid_price_imb2',
#        'reference_price_far_price_wap_imb2',
#        'reference_price_near_price_ask_price_imb2',
#        'reference_price_near_price_bid_price_imb2',
#        'reference_price_near_price_wap_imb2',
#        'reference_price_ask_price_bid_price_imb2',
#        'reference_price_ask_price_wap_imb2',
#        'reference_price_bid_price_wap_imb2',
#        'far_price_near_price_ask_price_imb2',
#        'far_price_near_price_bid_price_imb2', 'far_price_near_price_wap_imb2',
#        'far_price_ask_price_bid_price_imb2', 'far_price_ask_price_wap_imb2',
#        'far_price_bid_price_wap_imb2', 'near_price_ask_price_bid_price_imb2',
#        'near_price_ask_price_wap_imb2', 'near_price_bid_price_wap_imb2',
#        'ask_price_bid_price_wap_imb2',


#        'pca_prices', 'lgbm_preds']

stat_cols_to_drop = [
    "seconds_in_bucket",
    "imbalance_size",
    "imbalance_buy_sell_flag",
    "reference_price",
    "matched_size",
    "bid_price",
    "ask_price",
    "index_weight",
    "wap_calc",
    "initial_wap",
    "wap_weighted",
    "index_wap",
    "index_wap_init",
    "index_wap_move_to_init",
    "overall_medvol",
    "first5min_medvol",
    "last5min_medvol",
    "bid_plus_ask_sizes",
    "imbalance_ratio",
    "imb_s1",
    "imb_s2",
    "ask_x_size",
    "bid_x_size",
    "ask_minus_bid",
    "bid_price_over_ask_price",
    "reference_price_minus_far_price",
    "reference_price_times_far_price",
    "reference_price_times_near_price",
    "reference_price_minus_ask_price",
    "reference_price_times_ask_price",
    "reference_price_ask_price_imb",
    "reference_price_minus_bid_price",
    "reference_price_times_bid_price",
    "reference_price_bid_price_imb",
    "reference_price_minus_wap",
    "reference_price_times_wap",
    "reference_price_wap_imb",
    "far_price_minus_near_price",
    "far_price_times_near_price",
    "far_price_minus_ask_price",
    "far_price_times_ask_price",
    "far_price_minus_bid_price",
    "far_price_times_bid_price",
    "far_price_times_wap",
    "far_price_wap_imb",
    "near_price_minus_ask_price",
    "near_price_times_ask_price",
    "near_price_ask_price_imb",
    "near_price_minus_bid_price",
    "near_price_times_bid_price",
    "near_price_bid_price_imb",
    "near_price_minus_wap",
    "near_price_wap_imb",
    "ask_price_minus_bid_price",
    "ask_price_times_bid_price",
    "ask_price_minus_wap",
    "ask_price_times_wap",
    "ask_price_wap_imb",
    "bid_price_minus_wap",
    "bid_price_times_wap",
    "bid_price_wap_imb",
    "reference_price_far_price_near_price_imb2",
    "reference_price_far_price_ask_price_imb2",
    "reference_price_far_price_bid_price_imb2",
    "reference_price_far_price_wap_imb2",
    "reference_price_near_price_ask_price_imb2",
    "reference_price_near_price_bid_price_imb2",
    "reference_price_near_price_wap_imb2",
    "reference_price_ask_price_bid_price_imb2",
    "reference_price_ask_price_wap_imb2",
    "reference_price_bid_price_wap_imb2",
    "far_price_near_price_ask_price_imb2",
    "far_price_near_price_bid_price_imb2",
    "far_price_near_price_wap_imb2",
    "far_price_ask_price_bid_price_imb2",
    "far_price_ask_price_wap_imb2",
    "far_price_bid_price_wap_imb2",
    "near_price_ask_price_bid_price_imb2",
    "near_price_ask_price_wap_imb2",
    "near_price_bid_price_wap_imb2",
    "ask_price_bid_price_wap_imb2",

]

In [35]:
X["stats"] = np.split(
    np.nan_to_num(X[stat_cols].to_numpy(), nan=-1), indices_or_sections=len(X)
)

In [36]:
sys.getsizeof(X) / (1024**3)

4.189441833645105

In [37]:
X.pca_prices.value_counts().head(20)

pca_prices
-1.384228e+11    408604
-1.384228e+11    382508
-1.384228e+11    120018
-1.384228e+11    119660
-1.384228e+11    113200
-1.384228e+11    102242
-1.384228e+11     95361
-1.384228e+11     94760
-1.384228e+11     88780
-1.384228e+11     83781
-1.384228e+11     79486
-1.384228e+11     70315
-1.384228e+11     58804
-1.384228e+11     56950
-1.384228e+11     49809
-1.384228e+11     43103
-1.384228e+11     35132
-1.384228e+11     31165
-1.384228e+11     24685
-1.384228e+11     18211
Name: count, dtype: int64

In [38]:
X["wap_category"] = X["wap_price_t-60"].apply(
    lambda x: 0
    if x < -10
    else (
        1
        if x < -5
        else (
            2
            if x < -1.5
            else (3 if x < 1.5 else (4 if x < 5 else (5 if x < 10 else 6)))
        )
    )
)

In [39]:
X[["wap_category", "wap_price_t-60"]]

,wap_category,wap_price_t-60
0,1,-5.17
1,1,-5.29
2,2,-3.06
3,2,-3.41
4,3,0.56
...,...,...
4666363,6,10.05
4666364,4,4.36
4666365,5,5.80
4666366,2,-2.66


In [40]:
X["wap_category"] = pd.qcut(X["target"], q=5)

In [41]:
weights = (
    X["wap_category"].value_counts(sort=False).reset_index().sort_values("wap_category")
)

In [42]:
weights["norm_count"] = 1 - (weights["count"] / weights["count"].sum())
weights
weight = torch.tensor(weights["norm_count"].to_numpy(), device="cuda:0")
weight

tensor([0.7998, 0.7996, 0.8005, 0.7998, 0.8003], device='cuda:0',
       dtype=torch.float64)

In [43]:
means = X.groupby('wap_category')['target'].median().reset_index().reset_index(names='original_index').rename(columns={'target':'mean_target'})
means['cat_name'] = means['original_index'].astype(str)+means['wap_category'].astype(str)
means

C:\Users\Nick\AppData\Local\Temp\ipykernel_25096\1662138918.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  means = X.groupby('wap_category')['target'].median().reset_index().reset_index(names='original_index').rename(columns={'target':'mean_target'})


,original_index,wap_category,mean_target,cat_name
0,0,"(-385.291, -5.9]",-9.940267,"0(-385.291, -5.9]"
1,1,"(-5.9, -1.67]",-3.499985,"1(-5.9, -1.67]"
2,2,"(-1.67, 1.56]",-0.050068,"2(-1.67, 1.56]"
3,3,"(1.56, 5.76]",3.390312,"3(1.56, 5.76]"
4,4,"(5.76, 387.779]",9.800196,"4(5.76, 387.779]"


In [44]:
softmax = nn.Softmax()
weight = torch.tensor(
    [
        1,
        1.1,
        1,
        1.1,
        1,
    ],
    device="cuda:0",
)
# torch.log(torch.log(weight))
# softmax(weight)

In [45]:
ohe = OneHotEncoder(sparse_output=False)
ohe_out = ohe.fit_transform(
    X["wap_category"].to_numpy().reshape(-1, 1),
)

In [46]:
X["wap_target_OHE"] = [x for x in ohe_out]
X["wap_target_OHE"]

0          [0.0, 1.0, 0.0, 0.0, 0.0]
1          [0.0, 0.0, 1.0, 0.0, 0.0]
2          [0.0, 0.0, 0.0, 1.0, 0.0]
3          [0.0, 0.0, 0.0, 1.0, 0.0]
4          [0.0, 0.0, 0.0, 1.0, 0.0]
                     ...            
4666363    [1.0, 0.0, 0.0, 0.0, 0.0]
4666364    [0.0, 1.0, 0.0, 0.0, 0.0]
4666365    [0.0, 1.0, 0.0, 0.0, 0.0]
4666366    [0.0, 0.0, 0.0, 1.0, 0.0]
4666367    [0.0, 0.0, 0.0, 1.0, 0.0]
Name: wap_target_OHE, Length: 4666368, dtype: object

In [47]:
importlib.reload(utils.torch_classes)
trading_data = utils.torch_classes.TradingData(X,means)
hidden_size = 64
# trading_data.generate_batches()

100%|██████████| 95232/95232 [06:20<00:00, 250.32it/s]


In [48]:
trading_data.generate_batches()

Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 314.24it/s]


In [49]:
# for i,stocks in enumerate(trading_data.stocksDict.values()):
#     if i==0:
#         continue
#     else:
#         stocks.data_daily = []
# trading_data.train_batches = []
# del train
# del X
# X = []
torch.cuda.empty_cache()
gc.collect()
# del pca_prices

0

In [50]:
# del pca, pca_prices_items

In [51]:
importlib.reload(utils.torch_classes)
importlib.reload(utils.training_testing)

<module 'utils.training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\utils\\training_testing.py'>

In [52]:
optim_dict = {
    "RMSProp": optim.RMSprop,
    "Adam": optim.Adam,
    "RAdam": optim.RAdam,
    "NAdam": optim.NAdam,
    "AdamW": optim.AdamW,
    "SGD": optim.SGD,
    "Rprop": optim.Rprop,
}

In [53]:
def model_pipeline(
    trading_df=trading_data, config=None, prev_model_file=None, prev_model_version=450
):
    trading_df = trading_data
    with wandb.init(project="Optviver_new", config=config, save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        wandb.define_metric("Accuracy", summary="max")
        config = wandb.config

        input_size = len(trading_df.stocksDict[0].data_daily[0][0])

        target_size_ohe = len(trading_df.stocksDict[0].target_daily_ohe[0][0])

        print(target_size_ohe)
        
        model = utils.torch_classes.GRUNetV3(
            input_size,
            config["hidden_size"],
            num_layers=config["num_layers"],
            fc0_size=config["fc0_size"],
            target_size=target_size_ohe
        ).to("cuda:0")

        wandb.watch(model, log='all') 
        optimizer = optim_dict[config['optim']](model.parameters(), lr=config['learning_rate'], weight_decay=0.001)
        
        config = wandb.config
        print(config)
        print(config['ohe_targets'])
        if prev_model_file != None:
            model_name = prev_model_file
            model_loc = f"models/{model_name}/{model_name}_{prev_model_version}.pt"
            model_data = torch.load(model_loc, map_location=torch.device("cuda:0"))
            print(model_data["model_state_dict"].keys())
            print(model_data.keys())

            del_keys = ['fc_final.weight', 'fc_final.bias', 'fc_wap0.weight']
            [model_data['model_state_dict'].pop(k) for k in del_keys]
            model.load_state_dict(model_data["model_state_dict"], strict=False)
            # optimizer.load_state_dict(model_data["optim"])

        print(model)
        trading_df.reset_hidden(
            hidden_size=config["hidden_size"], num_layers=config["num_layers"]
        )
        criterion = nn.CrossEntropyLoss()
        print(criterion)
        print(optimizer)
        output = utils.training_testing.train_model(
            trading_df, model, config, optimizer, criterion
        )

    return output


# arroios

In [59]:
importlib.reload(utils.torch_classes)
importlib.reload(utils.training_testing)

<module 'utils.training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\utils\\training_testing.py'>

In [55]:
config_static = {
    "learning_rate": 0.00008,
    "hidden_size": 128,
    "num_layers": 2,
    "batch_norm": 1,
    "epochs": 5000,
    "mini_batches": 400,
    "fc0_size": 256,
    "note": "Fixed hidden_test issue, cold start",
    'optim': 'RAdam',
    'ohe_targets':means,
}
config = config_static
torch.cuda.empty_cache()
trading_data.detach_hidden()

In [56]:
%env "CUDA_LAUNCH_BLOCKING" 1

env: "CUDA_LAUNCH_BLOCKING"=1


In [57]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [60]:
CUDA_LAUNCH_BLOCKING = 1
# output = model_pipeline(trading_data, config_static,)
output = model_pipeline(trading_data, config_static)

5
{'learning_rate': 8e-05, 'hidden_size': 128, 'num_layers': 2, 'batch_norm': 1, 'epochs': 5000, 'mini_batches': 400, 'fc0_size': 256, 'note': 'Fixed hidden_test issue, cold start', 'optim': 'RAdam', 'ohe_targets': '   original_index      wap_category  mean_target           cat_name\n0               0  (-385.291, -5.9]    -9.940267  0(-385.291, -5.9]\n1               1     (-5.9, -1.67]    -3.499985     1(-5.9, -1.67]\n2               2     (-1.67, 1.56]    -0.050068     2(-1.67, 1.56]\n3               3      (1.56, 5.76]     3.390312      3(1.56, 5.76]\n4               4   (5.76, 387.779]     9.800196   4(5.76, 387.779]'}
   original_index      wap_category  mean_target           cat_name
0               0  (-385.291, -5.9]    -9.940267  0(-385.291, -5.9]
1               1     (-5.9, -1.67]    -3.499985     1(-5.9, -1.67]
2               2     (-1.67, 1.56]    -0.050068     2(-1.67, 1.56]
3               3      (1.56, 5.76]     3.390312      3(1.56, 5.76]
4               4   (5.76, 38

  0%|          | 0/5000 [00:00<?, ?it/s]

created path


In [ ]:
# torch.cuda.memory._dump_snapshot("my_snapshot.pickle")

In [ ]:
torch.rand(10).to("cuda:0")

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
stop

In [ ]:
output

In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "val_epoch_loss", "goal": "minimize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"values": ["adamW", "adam", "SGD", "RMSprop"]},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "num_layers": {"values": [2]},
    "hidden_size": {"values": [128, 256, 512]},
    "learning_rate": {"values": [0.001, 0.0005, 0.0001, 0.00005, 0.00001]},
    "epochs": {"value": 500}
    # 'batch_norm':{'values':[0,1,2]}
}

sweep_config["parameters"] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project="Optiver Sweeps")
# CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Create sweep with ID: upepev1k
Sweep URL: https://wandb.ai/nickojelly/Optiver%20Sweeps/sweeps/upepev1k


wandb: Agent Starting Run: t85ko0dj with config:
wandb: 	epochs: 500
wandb: 	f0_layer_size: 128
wandb: 	f1_layer_size: 64
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.0001
wandb: 	num_layers: 2
wandb: 	optimizer: SGD
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Traceback (most recent call last):
  File "C:\Users\Nick\AppData\Local\Temp\ipykernel_18468\3034068021.py", line 10, in model_pipeline
    model = utils.torch_classes.GRUNetV3(input_size,config['hidden_size'],num_layers=config['num_layers']).to('cuda:0')
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\module.py", line 1160, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\module.py", line 810, in _apply
    module._apply(fn)
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\rnn.py", line 213, in _apply
    ret = super()._apply(fn, recurse)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\module.py", line 833, in _apply
    param_applied = fn(pa

wandb: Ctrl + C detected. Stopping sweep.
